In [1]:
import pandas as pd
import numpy as np

## Solution
Identifying where there needs to be more focus on local production of food (vegetables in this case).

## Steps


- Load population of per county in the USA

- Load production of vegetables per county in the USA

- Load average vegetables consumption per county.

- Subtract production and consumption. There will be surplus and deficit.

- Find a way to match surplus and deficit, and to penalize for the distance.

## Assumptions

- Yield per acre is uniform accross farms.

## Data Exploration

In [2]:
local = pd.read_excel('data/DataDownload.xls',sheetname='LOCAL')
pop = pd.read_excel('data/DataDownload.xls', sheetname='Supplemental Data - County')

In [3]:
local.head()

,FIPS,State,County,DIRSALES_FARMS07,PCT_LOCLFARM07,PCT_LOCLSALE07,DIRSALES07,PC_DIRSALES07,FMRKT09,FMRKT13,...,BERRY_ACRESPTH07,SLHOUSE07,GHVEG_FARMS07,GHVEG_SQFT07,GHVEG_SQFTPTH07,FOODHUB12,CSA07,AGRITRSM_OPS07,AGRITRSM_RCT07,FARM_TO_SCHOOL
0,1001,AL,Autauga,25.0,6.0,0.6,100.0,2.01,2.0,1.0,...,NaN,0.0,0.0,0.0,0.0,0.0,2.0,7.0,228000.0,0.0
1,1003,AL,Baldwin,80.0,7.0,0.7,715.0,4.16,4.0,4.0,...,0.457136,1.0,0.0,0.0,0.0,0.0,13.0,18.0,124000.0,0.0
2,1005,AL,Barbour,18.0,2.9,0.0,11.0,0.37,2.0,3.0,...,NaN,0.0,0.0,0.0,0.0,0.0,1.0,27.0,163000.0,0.0
3,1007,AL,Bibb,12.0,5.7,NaN,46.0,2.14,1.0,1.0,...,0.279265,0.0,0.0,0.0,0.0,0.0,2.0,5.0,NaN,0.0
4,1009,AL,Blount,84.0,5.9,0.3,429.0,7.59,1.0,1.0,...,0.492386,0.0,2.0,NaN,NaN,0.0,7.0,10.0,293000.0,0.0


In [4]:
local.columns

Index([u'FIPS', u'State', u'County', u'DIRSALES_FARMS07', u'PCT_LOCLFARM07',
       u'PCT_LOCLSALE07', u'DIRSALES07', u'PC_DIRSALES07', u'FMRKT09',
       u'FMRKT13', u'PCH_FMRKT_09_13', u'FMRKTPTH09', u'FMRKTPTH13',
       u'PCH_FMRKTPTH_09_13', u'FMRKT_SNAP13', u'PCT_FMRKT_SNAP13',
       u'FMRKT_WIC13', u'PCT_FMRKT_WIC13', u'FMRKT_WICCASH13',
       u'PCT_FMRKT_WICCASH13', u'FMRKT_SFMNP13', u'PCT_FMRKT_SFMNP13',
       u'FMRKT_FRVEG13', u'PCT_FRMKT_FRVEG13', u'FMRKT_ANMLPROD13',
       u'PCT_FRMKT_ANMLPROD13', u'FMRKT_OTHER13', u'PCT_FMRKT_OTHER13',
       u'VEG_FARMS07', u'VEG_ACRES07', u'VEG_ACRESPTH07', u'FRESHVEG_FARMS07',
       u'FRESHVEG_ACRES07', u'FRESHVEG_ACRESPTH07', u'ORCHARD_FARMS07',
       u'ORCHARD_ACRES07', u'ORCHARD_ACRESPTH07', u'BERRY_FARMS07',
       u'BERRY_ACRES07', u'BERRY_ACRESPTH07', u'SLHOUSE07', u'GHVEG_FARMS07',
       u'GHVEG_SQFT07', u'GHVEG_SQFTPTH07', u'FOODHUB12', u'CSA07',
       u'AGRITRSM_OPS07', u'AGRITRSM_RCT07', u'FARM_TO_SCHOOL'],
      dtype

In [5]:
pop.head()

,FIPS Code,State,County Name,"Population Estimate, 2007","Population Estimate, 2008","Population Estimate, 2009",2010 Census Population,"Population Estimate, 2011","Population Estimate, 2012"
0,1001,AL,Autauga,49834,50354,50756,54571,55267,55514
1,1003,AL,Baldwin,172815,176212,179878,182265,186717,190790
2,1005,AL,Barbour,29736,29836,29737,27457,27119,27201
3,1007,AL,Bibb,21485,21589,21587,22915,22766,22597
4,1009,AL,Blount,56866,57794,58345,57322,57677,57826


In [6]:
pop.columns

Index([u'FIPS Code', u'State', u'County Name', u'Population Estimate, 2007',
       u'Population Estimate, 2008', u'Population Estimate, 2009',
       u'2010 Census Population', u'Population Estimate, 2011',
       u'Population Estimate, 2012'],
      dtype='object')

## Data Processing

In [15]:
veg_farms = local[['FIPS','State', 'County','VEG_ACRES07', 'FRESHVEG_ACRES07', 'GHVEG_FARMS07']]

In [16]:
veg_farms[veg_farms['FRESHVEG_ACRES07'].isnull() & ~veg_farms['VEG_ACRES07'].isnull()]

,FIPS,State,County,VEG_ACRES07,FRESHVEG_ACRES07,GHVEG_FARMS07
0,1001,AL,Autauga,948.0,NaN,0.0
1,1003,AL,Baldwin,2280.0,NaN,0.0
3,1007,AL,Bibb,65.0,NaN,0.0
5,1011,AL,Bullock,134.0,NaN,0.0
6,1013,AL,Butler,80.0,NaN,0.0
9,1019,AL,Cherokee,115.0,NaN,0.0
16,1033,AL,Colbert,61.0,NaN,1.0
20,1041,AL,Crenshaw,42.0,NaN,0.0
21,1043,AL,Cullman,1329.0,NaN,2.0
24,1049,AL,DeKalb,1152.0,NaN,2.0


In [17]:
non_na_vegetables = veg_farms.dropna()

In [18]:
non_na_vegetables['VEG_ACRES07'].sum() #Acres harvested in 2007

3740457.0

In [19]:
tons_per_acre = 25727576./1784290. # Production in tons / Acres harvested - Source in references

In [20]:
non_na_vegetables['FRESHVEG_TONS07'] = non_na_vegetables['FRESHVEG_ACRES07'] * tons_per_acre 

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
non_na_vegetables.FIPS = non_na_vegetables.FIPS.astype(int)

In [71]:
vegetables = pd.DataFrame(veg_farms['FIPS'])
vegetables = pd.merge(vegetables, non_na_vegetables, on='FIPS', how='left')
vegetables = pd.merge(vegetables, pop, left_on='FIPS', right_on='FIPS Code')

In [72]:
vegetables.drop(vegetables.columns[[11,12,13,14,15]], inplace=True, axis=1)
vegetables.drop(['State_x', 'County','FIPS Code'], axis=1, inplace=True)

In [73]:
avg_per_capita_food_consumption = 0.2075

In [74]:
vegetables['VEG_CONSUMPTION_TONS07'] = vegetables['Population Estimate, 2007'] * avg_per_capita_food_consumption 

In [75]:
vegetables.FRESHVEG_TONS07.fillna(0.0, inplace=True)

In [76]:
vegetables['SURPLUS'] = vegetables['FRESHVEG_TONS07'] - vegetables['VEG_CONSUMPTION_TONS07']

In [77]:
vegetables.rename(columns={'FRESHVEG_TONS07' : 'VEG_PRODUCTION_TONS07'}, inplace=True)

In [78]:
vegetables

,FIPS,VEG_ACRES07,FRESHVEG_ACRES07,GHVEG_FARMS07,VEG_PRODUCTION_TONS07,State_y,County Name,"Population Estimate, 2007",VEG_CONSUMPTION_TONS07,SURPLUS
0,1001,NaN,NaN,NaN,0.000000,AL,Autauga,49834,10340.5550,-10340.555000
1,1003,NaN,NaN,NaN,0.000000,AL,Baldwin,172815,35859.1125,-35859.112500
2,1005,32.0,30.0,0.0,432.568293,AL,Barbour,29736,6170.2200,-5737.651707
3,1007,NaN,NaN,NaN,0.000000,AL,Bibb,21485,4458.1375,-4458.137500
4,1009,585.0,581.0,2.0,8377.405946,AL,Blount,56866,11799.6950,-3422.289054
5,1011,NaN,NaN,NaN,0.000000,AL,Bullock,10966,2275.4450,-2275.445000
6,1013,NaN,NaN,NaN,0.000000,AL,Butler,20243,4200.4225,-4200.422500
7,1015,46.0,38.0,0.0,547.919838,AL,Calhoun,112808,23407.6600,-22859.740162
8,1017,39.0,39.0,1.0,562.338781,AL,Chambers,34731,7206.6825,-6644.343719
9,1019,NaN,NaN,NaN,0.000000,AL,Cherokee,24474,5078.3550,-5078.355000


In [80]:
vegetables.to_csv('data/surplus.csv', index=False)